In [1]:
# alternate way
# id = item description
# repo = repository
# 1st step is lookup from repo
# 2nd step is string match from short master
# 3rd step is string match from main master
# please mind the working directory at the time of reading the files  

import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz

# reading T-1 data, this is the mysql query output and is at awb level
id = pd.read_csv(r"C:\Users\24352\Desktop\categorisation1\id.csv",encoding='latin-1') 

# reading short master
id_master_short = pd.read_csv(r"C:\Users\24352\Desktop\categorisation1\Category_Master_Short.csv",encoding='latin-1')

# previous successful categorisations repository with unique item descriptions
id_repo = pd.read_csv(r"C:\Users\24352\Desktop\categorisation1\ID_Repo.csv",encoding='latin-1')

from nltk.corpus import stopwords

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# creating a new data frame with unique item descriptions on T-1 data
# this is the data frame which will be used later on for string match and exact match look up
# this is done to reduce the time complexity

id_unique = pd.DataFrame(id["item_description"].unique(),columns=['item_description'])
print(id.shape)# prints the count of volume
print(id_unique.shape)# prints the count of unique id in t-1 volume
print(id_repo.shape)# prints the count of unique ids in id repository 

(101, 4)
(98, 1)
(1, 5)


In [3]:
# subset of short master containing only keywords
id_master_sub = id_master_short[["L4 Category"]]

In [4]:
# removing the blank values
id_master_sub = id_master_sub[id_master_sub["L4 Category"].notnull()]

In [5]:
# function to clean the ids
def rmstop(x):
    #pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    #x = pattern.sub('', x)
    x = str(x).lower()
    x = re.sub(r'[^a-zA-Z0-9\s]', ' ', x)
    return(x)

In [6]:
# converting to string type
id_unique = id_unique.astype(str)

In [7]:
# converting to string type
id_master_sub = id_master_sub.astype(str)

In [8]:
# viewing the head of id repo
id_repo.head()

,item_description,L1 Category,L2 Category,L3 Category,L4 Category
0,100FIT Tempered Glass for Honor 9 lite (11D)-...,"Mobiles, Tablets, Mobile Accessories",Screen Protectors,Hardened Glass,Tempered glass


In [9]:
# merging the the two data frames to look up from idrepo
lookup = pd.merge(id_unique,id_repo,how="left",left_on="item_description",right_on="item_description")

In [10]:
# subset of successful exact match
lookup_yes = lookup[lookup["L4 Category"].notnull()]
lookup_yes.head()

,item_description,L1 Category,L2 Category,L3 Category,L4 Category
97,100FIT Tempered Glass for Honor 9 lite (11D)-...,"Mobiles, Tablets, Mobile Accessories",Screen Protectors,Hardened Glass,Tempered glass


In [11]:
# subset of successful exact match
lookup_no = lookup[lookup["L4 Category"].isnull()]
lookup_no.head()

,item_description,L1 Category,L2 Category,L3 Category,L4 Category
0,"TECHNICAL SELLER in-Ear Wired Earphone, 1.2m ...",NaN,NaN,NaN,NaN
1,Sobo Slim Filter WP 408H Hang On Filter Flow ...,NaN,NaN,NaN,NaN
2,Fabiano Appliances G-400El(Bfy) Designer Glas...,NaN,NaN,NaN,NaN
3,Faces: A chilling thriller of loyalty and bet...,NaN,NaN,NaN,NaN
4,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,NaN,NaN,NaN,NaN


In [12]:
# keeping the first match row only
lookup_yes = lookup_yes.groupby(['item_description'])['L1 Category','L2 Category','L3 Category','L4 Category'].first().reset_index()

In [13]:
# new dataframe with first stage completed
lookup_yes.head()
final_1 = pd.merge(id,lookup_yes,how="left",left_on= "item_description",right_on="item_description")

In [14]:
# count of unsuccessful matches in total volume
final_1["L4 Category"].isnull().value_counts()

True     100
False      1
Name: L4 Category, dtype: int64

In [15]:
# coverage percentage of this stage
final_1["L4 Category"].isnull().value_counts()[0]/final_1.shape[0]

0.009900990099009901

In [16]:
# subset of non match id from the look upstage
non_match = final_1[final_1["L4 Category"].isnull()]

In [17]:
# 1st step final data frame
final_1 = final_1[final_1["L4 Category"].notnull()]

In [18]:
non_match.head()

,airwaybill_number,item_description,inscan_date,shipper_id,L1 Category,L2 Category,L3 Category,L4 Category
0,2013989109,"TECHNICAL SELLER in-Ear Wired Earphone, 1.2m ...",8/13/2020 0:00,2272,NaN,NaN,NaN,NaN
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,NaN,NaN,NaN,NaN
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,NaN,NaN,NaN,NaN
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,NaN,NaN,NaN,NaN
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,NaN,NaN,NaN,NaN


In [19]:
# non match unique id data frame
id_unique = pd.DataFrame(non_match["item_description"].unique(),columns=['item_description'])
id_unique = id_unique.astype(str)

In [20]:
# cleaning
id["item_description_2"] = id["item_description"].apply(rmstop)
id_unique["item_description_2"] = id_unique["item_description"].apply(rmstop)
id_master_sub["L4_Category_2"] = id_master_sub["L4 Category"].apply(rmstop)
id_master_short["L4_Category_2"] = id_master_short["L4 Category"].apply(rmstop)

In [21]:
# creating dataframe with all cominations 
from itertools import product
#list(product(id['item_description'], id_master['Sub_Category']))
df = pd.DataFrame(data=list(product(id_unique['item_description_2'], id_master_sub['L4_Category_2'])), columns=['Item_description','L4_Category']) 
df.shape 

(3977, 2)

In [22]:
# checkpoint  
print("ID_Repo observations",id_repo.shape[0])
print("Vol to be categorised",id.shape[0])
print("Unique ID count",lookup.shape[0]) # this is done because id_unique keeps changing
print("ID match from ID_Repo",lookup_yes.shape[0])
print("ID to be categorised",id_unique.shape[0])
print("ID to be categorised_check",lookup_no.shape[0])

ID_Repo observations 1
Vol to be categorised 101
Unique ID count 98
ID match from ID_Repo 1
ID to be categorised 97
ID to be categorised_check 97


In [23]:
# string match
df["token_set_ratio"]  = df.apply(lambda x: fuzz.token_set_ratio(x['Item_description'], x['L4_Category']), axis=1)

In [24]:
# sorting the dataframe in highest to lowest order
df = df.sort_values(by=['token_set_ratio'], ascending=False)

In [25]:
# viewing the dataframe
df.head()

,Item_description,L4_Category,token_set_ratio
2419,printbharat men s women s regular fit t shi...,t shirt,100
3380,blaxstoc magnetic bluetooth earphone wireless...,mobile phone,100
3364,blaxstoc magnetic bluetooth earphone wireless...,earphone,100
3214,xusive leather wallet flip book cover case fo...,wallet,100
2428,printbharat men s women s regular fit t shi...,shirt,100


In [26]:
# subset of only greater than 90% match
df2 = df[df["token_set_ratio"]>=90].copy() 

In [27]:
# converting to string type
df2["Item_description"] = df2["Item_description"].astype(str)
df2["L4_Category"] = df2["L4_Category"].astype(str)

# right.groupby(['age'])['salary'].first().reset_index()

# lengths calculated to single out the longest match
df2["len_id"] = df2["Item_description"].str.len()
df2["len_sc"] = df2["L4_Category"].str.len()
df2["len_check"] = np.where(df2["len_id"]>df2["len_sc"],1,0)

In [28]:
# sorting the data frame
df2 = df2.sort_values(['len_id', 'len_sc','len_check'], ascending=[False, False, False])

In [29]:
# selecting the first row only thereby keeping the longest match 
df2 = df2.groupby(['Item_description'])['L4_Category'].first().reset_index()

In [30]:
df2.head()

,Item_description,L4_Category
0,blaxstoc magnetic bluetooth earphone wireless...,mobile phone
1,blaxstoc wireless bluetooth earphone headphon...,headphone
2,liberty coolers cool99 120 men s formal slipp...,slippers
3,micamount rajah ayurveda rootz herbal shampoo...,shampoo
4,nine9 sh12 wireless bluetooth headphone for a...,headphone


In [31]:
# cleaning the id in the dataframe left after 1st step
non_match["item_description_2"] = non_match["item_description"].apply(rmstop)

In [32]:
# dropping the columns to replicate the query output data
non_match.drop(["L1 Category","L2 Category","L3 Category","L4 Category"],axis=1, inplace=True)

In [33]:
# viewing the data frame
non_match.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2
0,2013989109,"TECHNICAL SELLER in-Ear Wired Earphone, 1.2m ...",8/13/2020 0:00,2272,technical seller in ear wired earphone 1 2m ...
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...


In [34]:
# look up the results/keywords of the step 2 string match 
final_2 = pd.merge(non_match,df2,how="left",left_on="item_description_2",right_on="Item_description")

In [35]:
final_2.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2,Item_description,L4_Category
0,2013989109,"TECHNICAL SELLER in-Ear Wired Earphone, 1.2m ...",8/13/2020 0:00,2272,technical seller in ear wired earphone 1 2m ...,technical seller in ear wired earphone 1 2m ...,earphone
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...,NaN,NaN
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...,NaN,NaN
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...,NaN,NaN
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...,NaN,NaN


In [36]:
# look up from the short master(BOW)
final_2 = pd.merge(final_2,id_master_short,how="left", left_on= "L4_Category",right_on="L4_Category_2")

In [37]:
# dropping the columns that are not required
final_2.drop(["L4_Category","L4_Category_2","Item_description"],axis=1,inplace=True)
final_2.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2,L1 Category,L2 Category,L3 Category,L4 Category
0,2013989109,"TECHNICAL SELLER in-Ear Wired Earphone, 1.2m ...",8/13/2020 0:00,2272,technical seller in ear wired earphone 1 2m ...,"Laptops, Electronics and Accessories",Headphones,EarPhones,Earphone
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...,NaN,NaN,NaN,NaN
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...,NaN,NaN,NaN,NaN
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...,NaN,NaN,NaN,NaN
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...,NaN,NaN,NaN,NaN


In [38]:
# count of unsuccessful matches in remaining volume
final_2["L4 Category"].isnull().value_counts()

True     86
False    14
Name: L4 Category, dtype: int64

In [39]:
non_match.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2
0,2013989109,"TECHNICAL SELLER in-Ear Wired Earphone, 1.2m ...",8/13/2020 0:00,2272,technical seller in ear wired earphone 1 2m ...
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...


In [40]:
# subset of output with no match after step2
non_match = final_2[final_2["L4 Category"].isnull()]

# subset of successfull match after step 2
final_2 = final_2[final_2["L4 Category"].notnull()]

In [41]:
# checkpoint
print("Final_1",final_1.shape[0])
print("Final_2",final_2.shape[0])
print("Non_match",non_match.shape[0])
print("Total Observations",final_1.shape[0]+final_2.shape[0]+non_match.shape[0])
print("Total Observations at start",id.shape[0])

Final_1 1
Final_2 14
Non_match 86
Total Observations 101
Total Observations at start 101


In [42]:
# dropping the columns not required for now
non_match.drop(["L1 Category","L2 Category","L3 Category","L4 Category"],axis=1, inplace=True)

In [43]:
non_match.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...
5,2013952969,PRIDHI 140 TC Cotton Single Bedsheet with 1 P...,8/13/2020 0:00,2272,pridhi 140 tc cotton single bedsheet with 1 p...


In [44]:
# creating a dataframe with unique ids left after the completion of stage 2  
id_unique = pd.DataFrame(non_match["item_description"].unique(),columns=['item_description'])
id_unique = id_unique.astype(str)

In [45]:
# reading the main master for categorisation
# mind the working directory here
id_master = pd.read_csv(r"C:\Users\24352\Desktop\categorisation1\Category_Master.csv",encoding='latin-1')

In [46]:
id_master.head()

,L1 Category,L2 Category,L3 Category,L4 Category
0,Sports & Fitness,Fitness Accessories,Exercise bands,Ab Exerciser
1,Sports & Fitness,Fitness Accessories,Exercise bands,Abs Exerciser
2,Games & Toys Store and Baby Products,Baby Toys,Toys,Activity Kit
3,"Mobiles, Tablets, Mobile Accessories",Cables and Chargers,Chargers,adapter
4,Home and Decor,Door Locks and Hardware,Hooks,Adhesive Products


In [47]:
# master dimensions
id_master.shape

(980, 4)

In [48]:
# subsetting keywords from master
id_master_sub = id_master[["L4 Category"]]

In [49]:
# removing the blanks
id_master_sub = id_master_sub[id_master_sub["L4 Category"].notnull()]

In [50]:
id_master_sub = id_master_sub.astype(str)

In [51]:
id_master.head()

,L1 Category,L2 Category,L3 Category,L4 Category
0,Sports & Fitness,Fitness Accessories,Exercise bands,Ab Exerciser
1,Sports & Fitness,Fitness Accessories,Exercise bands,Abs Exerciser
2,Games & Toys Store and Baby Products,Baby Toys,Toys,Activity Kit
3,"Mobiles, Tablets, Mobile Accessories",Cables and Chargers,Chargers,adapter
4,Home and Decor,Door Locks and Hardware,Hooks,Adhesive Products


In [52]:
# cleaning the columns to be used
id_unique["item_description_2"] = id_unique["item_description"].apply(rmstop)
id_master_sub["L4_Category_2"] = id_master_sub["L4 Category"].apply(rmstop)
id_master["L4_Category_2"] = id_master["L4 Category"].apply(rmstop)

In [53]:
# creating dataframe with all cominations 
from itertools import product
#list(product(id['item_description'], id_master['Sub_Category']))
df = pd.DataFrame(data=list(product(id_unique['item_description_2'], id_master_sub['L4_Category_2'])), columns=['Item_description','L4_Category']) 
df.shape

(43775, 2)

In [54]:
# string match function
df["token_set_ratio"]  = df.apply(lambda x: fuzz.token_set_ratio(x['Item_description'], x['L4_Category']), axis=1)

In [55]:
# subsetting greater than eqaul to 90% match 
df2 = df[df["token_set_ratio"]>=90].copy() 

In [56]:
# converting to string type
df2["Item_description"] = df2["Item_description"].astype(str)
df2["L4_Category"] = df2["L4_Category"].astype(str)

# right.groupby(['age'])['salary'].first().reset_index()

# length calculated to single out the longest match
df2["len_id"] = df2["Item_description"].str.len()
df2["len_sc"] = df2["L4_Category"].str.len()
df2["len_check"] = np.where(df2["len_id"]>df2["len_sc"],1,0)

In [57]:
# sorting the data frame  and selecting the first row to keep the longest match
df2 = df2.sort_values(['len_id', 'len_sc','len_check'], ascending=[False, False, False])
df2 = df2.groupby(['Item_description'])['L4_Category'].first().reset_index()
df2.head()

,Item_description,L4_Category
0,aadya life sciences llp chloasma care cream s...,skin care
1,aarnik version 3 0 heavy duty shockproof armo...,oppo
2,abony cotton hand towel 40x60 cm white,towel
3,barry john 10 w 3 inch speaker for home theat...,speaker
4,bluedot present designer printed 6 piece cott...,pillow


In [58]:
non_match.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2
1,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...
2,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...
3,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...
4,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...
5,2013952969,PRIDHI 140 TC Cotton Single Bedsheet with 1 P...,8/13/2020 0:00,2272,pridhi 140 tc cotton single bedsheet with 1 p...


In [59]:
# look up of the step 3 keywords/results to awb level data
final_3 = pd.merge(non_match,df2,how="left",left_on="item_description_2",right_on="Item_description")
final_3.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2,Item_description,L4_Category
0,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...,NaN,NaN
1,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...,fabiano appliances g 400el bfy designer glas...,gas stove
2,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...,NaN,NaN
3,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...,festival home furnishings polycotton 400tc di...,diwan set
4,2013952969,PRIDHI 140 TC Cotton Single Bedsheet with 1 P...,8/13/2020 0:00,2272,pridhi 140 tc cotton single bedsheet with 1 p...,pridhi 140 tc cotton single bedsheet with 1 p...,bedsheet


In [60]:
# look up to get the main master categories to the awb level data
final_3 = pd.merge(final_3,id_master,how="left", left_on= "L4_Category",right_on="L4_Category_2")

In [61]:
# dropping the columns not required
final_3.drop(["L4_Category","L4_Category_2","Item_description"],axis=1,inplace=True)

In [62]:
final_3.head()

,airwaybill_number,item_description,inscan_date,shipper_id,item_description_2,L1 Category,L2 Category,L3 Category,L4 Category
0,2013950227,Sobo Slim Filter WP 408H Hang On Filter Flow ...,8/13/2020 0:00,2272,sobo slim filter wp 408h hang on filter flow ...,NaN,NaN,NaN,NaN
1,2013899399,Fabiano Appliances G-400El(Bfy) Designer Glas...,8/13/2020 0:00,2272,fabiano appliances g 400el bfy designer glas...,Kitchen and Dining,Cookware,Cooking Stoves,Gas stove
2,2013987606,Faces: A chilling thriller of loyalty and bet...,8/13/2020 0:00,2272,faces a chilling thriller of loyalty and bet...,NaN,NaN,NaN,NaN
3,2013977309,FESTIVAL HOME FURNISHINGS Polycotton 400TC Di...,8/13/2020 0:00,2272,festival home furnishings polycotton 400tc di...,Home and Decor,Bedroom Linen,Bedding Sets,Diwan set
4,2013952969,PRIDHI 140 TC Cotton Single Bedsheet with 1 P...,8/13/2020 0:00,2272,pridhi 140 tc cotton single bedsheet with 1 p...,Home and Decor,Bedroom Linen,Bedsheets,bedsheet


In [63]:
# count of unsuccessful matches in remaining volume
final_3["L4 Category"].isnull().value_counts()

False    55
True     33
Name: L4 Category, dtype: int64

In [64]:
# dropping the columns not required
final_3.drop(["item_description_2"],axis=1,inplace=True)
final_2.drop(["item_description_2"],axis=1,inplace=True)

In [65]:
# binding the the outputs of the 3 steps to create final output
final = pd.concat([final_1,final_2,final_3],axis=0)

In [66]:
# exporting the categorised volume to the working directory
final.to_csv("result_pyhton_id_final.csv",index=False)

In [67]:
# count of unsuccessful categorisation
final["L4 Category"].isnull().value_counts()

False    70
True     33
Name: L4 Category, dtype: int64

In [68]:
# checking for any null values. the sum here should always be zero
sum(final["L4 Category"]=="nan")

0

In [69]:
# coverage achieved
final["L4 Category"].isnull().value_counts()[0]/final.shape[0]

0.6796116504854369

In [70]:
# creating a subset of successful string match ids 
repo = final[final["L4 Category"].notnull()]

In [71]:
repo.shape

(70, 8)

In [72]:
# removing duplicate ids in repo created in the previous step
repo = repo.groupby(['item_description'])['L1 Category','L2 Category','L3 Category','L4 Category'].first().reset_index()

In [73]:
repo.shape

(65, 5)

In [74]:
# checkpoint, should be all true
# id repo is the repository contaning past ids categorised
repo.columns==id_repo.columns

array([ True,  True,  True,  True,  True])

In [75]:
# checking for any null values to be sure
repo["L4 Category"].isnull().value_counts()

False    65
Name: L4 Category, dtype: int64

In [76]:
# t-1 categorisation repo exported 
repo.to_csv("repo_current.csv",index=False)

In [77]:
# updating the id repo with t-1 successful categorisation
id_repo = pd.concat([id_repo,repo],axis=0)

In [78]:
id_repo.shape

(66, 5)

In [79]:
# removing duplicate ids if any 
id_repo = id_repo.groupby(['item_description'])['L1 Category','L2 Category','L3 Category','L4 Category'].first().reset_index()

In [80]:
id_repo.shape

(65, 5)

In [81]:
# exporting the updated id_repo
id_repo.to_csv("ID_Repo.csv", index=False)